#### Génération d'un index "point" spatial sous Neo4j pour tous les objets

Objectifs :
- Génération d'un label ROAD_POINT pour les points routes
- Génération d'un index par code insee pour les communes (afin d'optimiser les requêtes sur les communes)
- Génération d'un index spatial pour tous les objets CITY, ROAD_POINT, (et NATIONAL ROAD_POINT)

In [4]:
from neo4j import GraphDatabase, basic_auth

Requêtes Neo4j :

In [2]:
def create_labels_road_points(tx):

    query = "MATCH (n) WHERE ToString(n.street_count)=n.street_count \
            SET n:ROAD_POINT RETURN count(n)"
    
    result = tx.run(query)
    return result.data()

In [3]:
def create_insee_index(tx):

    query = "  CREATE INDEX insee IF NOT EXISTS \
            FOR (c:CITY) \
            ON c.insee"

    result = tx.run(query)
    return result.data()

In [4]:
def create_point_location(tx, entity):

    query = "  MATCH (n1:" + entity + ") \
            SET n1.location = point({longitude:n1.x, latitude:n1.y}) \
            RETURN count(n1) AS count"

    result = tx.run(query)
    return result.data()

In [5]:
def change_type_string_integer(tx, entity):

    query = "  MATCH (n:" + entity + ") \
            SET n.insee = ToInteger(n.insee) \
            RETURN count(n) AS count"

    result = tx.run(query)
    return result.data()

In [6]:
def change_type_string_float_integer_RELATED(tx):

    query = "  MATCH ()-[r:RELATED_TO]-() \
            SET r.speed_kph = ToFloat(r.speed_kph) \
            SET r.speed_kph = ToFloat(r.speed_kph) \
            RETURN count(r) AS count"

    result = tx.run(query)
    return result.data()

In [7]:
def change_type_string_float(tx, entity):

    query = "  MATCH (n1:" + entity + ") \
            SET n1.x = toFloat(n1.x) \
            SET n1.y = toFloat(n1.y) \
            RETURN count(n1) AS count"

    result = tx.run(query)
    return result.data()

In [8]:
def create_index_communes(tx):
    query = "  CREATE POINT INDEX city_point_index IF NOT EXISTS \
            FOR (n:CITY) \
            ON n.location "

    result = tx.run(query)
    return result.data()

In [9]:
def create_index_road_point(tx):
    query = "  CREATE POINT INDEX road_point_index IF NOT EXISTS \
            FOR (n:ROAD_POINT) \
            ON n.location "

    result = tx.run(query)
    return result.data()

In [1]:
def convert_travel_time_RELATED(tx):
    query = "  MATCH ()-[r:RELATED]-() \
            SET r.travel_time = toInteger(r.travel_time) \
            RETURN count(*) AS count "

    result = tx.run(query)
    return result.data()

In [5]:
""" def create_spatial_index_national_point(tx):

    query = "  MATCH ()-[r:LINKED_TO]-() \
            SET r.location = point({longitude:r.x_1, latitude:r.y_1}) \
            RETURN count(r) AS count"

    result = tx.run(query)
    return result.data() """

In [6]:
""" def create_spatial_index_national_point_2(tx):

    query = "  MATCH ()-[r:LINKED_TO]-() \
            SET r.location_2 = point({longitude:r.x_2, latitude:r.y_2}) \
            RETURN count(r) AS count"

    result = tx.run(query)
    return result.data() """

In [7]:
""" def create_spatial_index_national_point_center(tx):

    query = "  MATCH ()-[r:LINKED_TO]-() \
            SET r.location_center = point({longitude:r.x_center, latitude:r.y_center}) \
            RETURN count(r) AS count"

    result = tx.run(query)
    return result.data() """

In [2]:
PASSWORD_NEO4J = 'passwordneo4j'

In [5]:
driver = GraphDatabase.driver(
  "bolt://localhost:7687",
  auth=basic_auth("neo4j", PASSWORD_NEO4J))

Label generation for road points :

In [13]:
with driver.session() as session:

    result = session.execute_write(create_labels_road_points)

driver.close()

In [14]:
result[0]['count(n)']

523308

Index generation :

Add index from insee value for communes (de type RANGE INDEX):

In [15]:
with driver.session() as session:

    count_list = []

    result = session.execute_write(create_insee_index)

driver.close()

Suivre la création de l'index en exécutant une requête "SHOW INDEX" depuis le navigateur...

Autre besoin : convertir les insee des communes en type "Integer" (afin d'optimiser la recherche indexée)

In [16]:
with driver.session() as session:

    count_list = []

    result = session.execute_write(change_type_string_integer, 'CITY')

driver.close()

nb_nodes = result[0]['count']
                
print(f'Total Numbers of conversion: {nb_nodes}')

Total Numbers of conversion: 34460


Add spatial index for communes (type POINT INDEX):

En 2 requêtes (la première sert à créer les points "location")...

In [17]:
with driver.session() as session:

    count_list = []

    result = session.execute_write(create_point_location, 'CITY')

driver.close()

nb_nodes = result[0]['count']
                
print(f'Total Numbers of points created : {nb_nodes}')

Total Numbers of points created : 34460


In [18]:
with driver.session() as session:

    count_list = []

    result = session.execute_write(create_index_communes)

driver.close()

Suivre la création de l'index en exécutant une requête "SHOW INDEX" depuis le navigateur...

Add index for road_point :

Before that, we must convert x and y coords to float...

In [20]:
with driver.session() as session:

    count_list = []

    result = session.execute_write(change_type_string_float, 'ROAD_POINT')

driver.close()

nb_points = result[0]['count']
                
print(f'Total Numbers of conversion : {nb_points}')

Total Numbers of conversion : 523308


Ajout d'un index de type spatial (type POINT INDEX) aux points routes :

En 2 requêtes (la première sert à créer les points "location")...

In [21]:
with driver.session() as session:

    count_list = []

    result = session.execute_write(create_point_location, 'ROAD_POINT')

driver.close()

nb_points = result[0]['count']
                
print(f'Total Numbers of points created : {nb_points}')

Total Numbers of points created : 523308


In [22]:
with driver.session() as session:

    count_list = []

    result = session.execute_write(create_index_road_point)

driver.close()

Suivre la création de l'index en exécutant une requête "SHOW INDEX" depuis le navigateur...

Dernière chose : convertir les "travel_time" en type Integer (pour les algoritmes qui vont suivre) :
(le temps de trajet est donc ici arrondi à la minute)

In [6]:
with driver.session() as session:

    count_list = []

    result = session.execute_write(convert_travel_time_RELATED)

driver.close()

nb_points = result[0]['count']
                
print(f'Total Numbers of conversion : {nb_points}')

Total Numbers of conversion : 2132278


(dans le résultat affiché, il y a deux fois plus de résultats que de relations... tout est normal !)

---

OPTIONNEL : Création des index pour les points NATIONAL_POINT (non utilisés)

Moreover, We must create index for road points... first, the edges :

In [9]:
"""with driver.session() as session:

    count_list = []

    result = session.execute_write(create_spatial_index_national_point)

driver.close() """

In [12]:
""" with driver.session() as session:

    count_list = []

    result = session.execute_write(create_spatial_index_national_point_2)

driver.close() """

In [14]:
""" with driver.session() as session:

    count_list = []

    result = session.execute_write(create_spatial_index_national_point_center)

driver.close() """

Add spatial index for National Road Points :

In [10]:
"""  with driver.session() as session:

    count_list = []

    result = session.execute_write(create_spatial_index, 'NATIONAL_ROAD_POINT')

driver.close()

nb_points = result[0]['count']
                
print(f'Total Numbers of index created : {nb_points}') """

Total Numbers of index created : 6309
